<a href="https://colab.research.google.com/github/BrokenShell/DS-Unit-1-Sprint-1-Data-Wrangling-and-Storytelling/blob/master/module3-join-and-reshape-data/LS_DS_113_Join_and_Reshape_Data_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 1, Sprint 1, Module 3*

---

# Join and Reshape datasets

Objectives
- concatenate data with pandas
- merge data with pandas
-  understand tidy data formatting
-  melt and pivot data with pandas

Links
- [Pandas Cheat Sheet](https://github.com/pandas-dev/pandas/blob/master/doc/cheatsheet/Pandas_Cheat_Sheet.pdf)
- [Tidy Data](https://en.wikipedia.org/wiki/Tidy_data)
  - Combine Data Sets: Standard Joins
  - Tidy Data
  - Reshaping Data
- Python Data Science Handbook
  - [Chapter 3.6](https://jakevdp.github.io/PythonDataScienceHandbook/03.06-concat-and-append.html), Combining Datasets: Concat and Append
  - [Chapter 3.7](https://jakevdp.github.io/PythonDataScienceHandbook/03.07-merge-and-join.html), Combining Datasets: Merge and Join
  - [Chapter 3.8](https://jakevdp.github.io/PythonDataScienceHandbook/03.08-aggregation-and-grouping.html), Aggregation and Grouping
  - [Chapter 3.9](https://jakevdp.github.io/PythonDataScienceHandbook/03.09-pivot-tables.html), Pivot Tables
  
Reference
- Pandas Documentation: [Reshaping and Pivot Tables](https://pandas.pydata.org/pandas-docs/stable/reshaping.html)
- Modern Pandas, Part 5: [Tidy Data](https://tomaugspurger.github.io/modern-5-tidy.html)

In [0]:
!wget https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz

--2019-10-30 21:18:59--  https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.240.6
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.240.6|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 205548478 (196M) [application/x-gzip]
Saving to: ‘instacart_online_grocery_shopping_2017_05_01.tar.gz.1’

instacart_online_gr 100%[===================>] 196.03M  74.8MB/s    in 2.6s    

2019-10-30 21:19:02 (74.8 MB/s) - ‘instacart_online_grocery_shopping_2017_05_01.tar.gz.1’ saved [205548478/205548478]



In [0]:
!tar --gunzip --extract --verbose --file=instacart_online_grocery_shopping_2017_05_01.tar.gz

instacart_2017_05_01/
instacart_2017_05_01/._aisles.csv
instacart_2017_05_01/aisles.csv
instacart_2017_05_01/._departments.csv
instacart_2017_05_01/departments.csv
instacart_2017_05_01/._order_products__prior.csv
instacart_2017_05_01/order_products__prior.csv
instacart_2017_05_01/._order_products__train.csv
instacart_2017_05_01/order_products__train.csv
instacart_2017_05_01/._orders.csv
instacart_2017_05_01/orders.csv
instacart_2017_05_01/._products.csv
instacart_2017_05_01/products.csv


In [0]:
%cd instacart_2017_05_01

/content/instacart_2017_05_01


In [0]:
!ls -lh *.csv

-rw-r--r-- 1 502 staff 2.6K May  2  2017 aisles.csv
-rw-r--r-- 1 502 staff  270 May  2  2017 departments.csv
-rw-r--r-- 1 502 staff 551M May  2  2017 order_products__prior.csv
-rw-r--r-- 1 502 staff  24M May  2  2017 order_products__train.csv
-rw-r--r-- 1 502 staff 104M May  2  2017 orders.csv
-rw-r--r-- 1 502 staff 2.1M May  2  2017 products.csv


# Assignment

## Join Data Practice

These are the top 10 most frequently ordered products. How many times was each ordered? 

1. Banana
2. Bag of Organic Bananas
3. Organic Strawberries
4. Organic Baby Spinach 
5. Organic Hass Avocado
6. Organic Avocado
7. Large Lemon 
8. Strawberries
9. Limes 
10. Organic Whole Milk

First, write down which columns you need and which dataframes have them.

Next, merge these into a single dataframe.

Then, use pandas functions from the previous lesson to get the counts of the top 10 most frequently ordered products.

In [0]:
##### YOUR CODE HERE #####
import pandas as pd

In [0]:
df = pd.read_csv("products.csv", index_col="product_id").drop(['aisle_id', 'department_id'], axis=1)

In [0]:
orders = pd.read_csv("order_products__prior.csv").drop(['order_id', 'add_to_cart_order', 'reordered'], axis=1)
order_numbers = orders['product_id'].value_counts()
df['num_of_orders'] = [order_numbers[i] if i in order_numbers else 0 for i in df.index]

In [104]:
s = df.sort_values('num_of_orders', ascending=False)
print("Top 10 Products")
s.head(10)

Top 10 Products


,product_name,num_of_orders
product_id,,
24852,Banana,472565
13176,Bag of Organic Bananas,379450
21137,Organic Strawberries,264683
21903,Organic Baby Spinach,241921
47209,Organic Hass Avocado,213584
47766,Organic Avocado,176815
47626,Large Lemon,152657
16797,Strawberries,142951
26209,Limes,140627


## Reshape Data Section

- Replicate the lesson code
- Complete the code cells we skipped near the beginning of the notebook
- Table 2 --> Tidy
- Tidy --> Table 2
- Load seaborn's `flights` dataset by running the cell below. Then create a pivot table showing the number of passengers by month and year. Use year for the index and month for the columns. You've done it right if you get 112 passengers for January 1949 and 432 passengers for December 1960.

### Replicate the lesson code

In [0]:
from IPython.display import display, Image
url = 'https://cdn-images-1.medium.com/max/1600/1*vYGFQCafJtGBBX5mbl0xyw.png'
example = Image(url=url, width=600)

display(example)

In [0]:
import pandas as pd

In [0]:
order_products__prior = pd.read_csv('order_products__prior.csv').drop(['reordered'], axis=1)
print(order_products__prior.shape)
order_products__prior.head()

(32434489, 3)


,order_id,product_id,add_to_cart_order
0,2,33120,1
1,2,28985,2
2,2,9327,3
3,2,45918,4
4,2,30035,5


In [0]:
order_products__train = pd.read_csv('order_products__train.csv').drop(['reordered'], axis=1)
print(order_products__train.shape)
order_products__train.head()

(1384617, 3)


,order_id,product_id,add_to_cart_order
0,1,49302,1
1,1,11109,2
2,1,10246,3
3,1,49683,4
4,1,43633,5


In [0]:
orders = pd.read_csv('orders.csv').drop(['eval_set', 'days_since_prior_order'], axis=1)
print(orders.shape)
orders.head()

(3421083, 5)


,order_id,user_id,order_number,order_dow,order_hour_of_day
0,2539329,1,1,2,8
1,2398795,1,2,3,7
2,473747,1,3,3,12
3,2254736,1,4,4,7
4,431534,1,5,4,15


In [0]:
products = pd.read_csv('products.csv').drop(['aisle_id', 'department_id'], axis=1)
print(products.shape)
products.head()

(49688, 2)


,product_id,product_name
0,1,Chocolate Sandwich Cookies
1,2,All-Seasons Salt
2,3,Robust Golden Unsweetened Oolong Tea
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...
4,5,Green Chile Anytime Sauce


In [0]:
order_products = pd.concat([order_products__prior, order_products__train])

print(order_products.shape)
order_products.head()

(33819106, 3)


,order_id,product_id,add_to_cart_order
0,2,33120,1
1,2,28985,2
2,2,9327,3
3,2,45918,4
4,2,30035,5


In [0]:
condition = ((order_products['order_id'] == 2539329)  
          | (order_products['order_id'] == 2398795))

columns = ['order_id', 'product_id', 'add_to_cart_order']

order_products_subset = order_products.loc[condition, columns]
order_products_subset.head(11)

,order_id,product_id,add_to_cart_order
22742744,2398795,196,1
22742745,2398795,10258,2
22742746,2398795,12427,3
22742747,2398795,13176,4
22742748,2398795,26088,5
22742749,2398795,13032,6
24076664,2539329,196,1
24076665,2539329,14084,2
24076666,2539329,12427,3
24076667,2539329,26088,4


In [0]:
merged = pd.merge(orders_subset, 
                  order_products_subset, 
                  how='inner', 
                  on='order_id')
merged.head(11)

,user_id,order_id,order_number,order_dow,order_hour_of_day,product_id,add_to_cart_order
0,1,2539329,1,2,8,196,1
1,1,2539329,1,2,8,14084,2
2,1,2539329,1,2,8,12427,3
3,1,2539329,1,2,8,26088,4
4,1,2539329,1,2,8,26405,5
5,1,2398795,2,3,7,196,1
6,1,2398795,2,3,7,10258,2
7,1,2398795,2,3,7,12427,3
8,1,2398795,2,3,7,13176,4
9,1,2398795,2,3,7,26088,5


In [0]:
final = pd.merge(merged, 
                 products[['product_id', 'product_name']], 
                 how='left', 
                 on='product_id')
final.head(500)

,user_id,order_id,order_number,order_dow,order_hour_of_day,product_id,add_to_cart_order,product_name
0,1,2539329,1,2,8,196,1,Soda
1,1,2539329,1,2,8,14084,2,Organic Unsweetened Vanilla Almond Milk
2,1,2539329,1,2,8,12427,3,Original Beef Jerky
3,1,2539329,1,2,8,26088,4,Aged White Cheddar Popcorn
4,1,2539329,1,2,8,26405,5,XL Pick-A-Size Paper Towel Rolls
5,1,2398795,2,3,7,196,1,Soda
6,1,2398795,2,3,7,10258,2,Pistachios
7,1,2398795,2,3,7,12427,3,Original Beef Jerky
8,1,2398795,2,3,7,13176,4,Bag of Organic Bananas
9,1,2398795,2,3,7,26088,5,Aged White Cheddar Popcorn


In [0]:
final = final.sort_values(by=['order_number', 'add_to_cart_order'])
final

,user_id,order_id,order_number,order_dow,order_hour_of_day,product_id,add_to_cart_order,product_name
0,1,2539329,1,2,8,196,1,Soda
1,1,2539329,1,2,8,14084,2,Organic Unsweetened Vanilla Almond Milk
2,1,2539329,1,2,8,12427,3,Original Beef Jerky
3,1,2539329,1,2,8,26088,4,Aged White Cheddar Popcorn
4,1,2539329,1,2,8,26405,5,XL Pick-A-Size Paper Towel Rolls
5,1,2398795,2,3,7,196,1,Soda
6,1,2398795,2,3,7,10258,2,Pistachios
7,1,2398795,2,3,7,12427,3,Original Beef Jerky
8,1,2398795,2,3,7,13176,4,Bag of Organic Bananas
9,1,2398795,2,3,7,26088,5,Aged White Cheddar Popcorn


In [109]:
final.columns = [column.replace('_', ' ') for column in final]
final.head(11)
final = final[[
    'user id', 
    'order id',
    'order number', 
    'order dow', 
    'order hour of day', 
    'add to cart order',
    'product id',
    'product name',
]]
final

,user id,order id,order number,order dow,order hour of day,add to cart order,product id,product name
0,1,2539329,1,2,8,1,196,Soda
1,1,2539329,1,2,8,2,14084,Organic Unsweetened Vanilla Almond Milk
2,1,2539329,1,2,8,3,12427,Original Beef Jerky
3,1,2539329,1,2,8,4,26088,Aged White Cheddar Popcorn
4,1,2539329,1,2,8,5,26405,XL Pick-A-Size Paper Towel Rolls
5,1,2398795,2,3,7,1,196,Soda
6,1,2398795,2,3,7,2,10258,Pistachios
7,1,2398795,2,3,7,3,12427,Original Beef Jerky
8,1,2398795,2,3,7,4,13176,Bag of Organic Bananas
9,1,2398795,2,3,7,5,26088,Aged White Cheddar Popcorn


In [0]:
display(example)

### Complete the code cells we skipped near the beginning of the notebook

In [0]:
%matplotlib inline
import pandas as pd
import numpy as np
import seaborn as sns

table1 = pd.DataFrame(
    [[np.nan, 2],
     [16,    11], 
     [3,      1]],
    index=['John Smith', 'Jane Doe', 'Mary Johnson'], 
    columns=['treatmenta', 'treatmentb'])

table2 = table1.T

In [0]:
table1

,treatmenta,treatmentb
John Smith,NaN,2
Jane Doe,16.0,11
Mary Johnson,3.0,1


In [0]:
table2

,John Smith,Jane Doe,Mary Johnson
treatmenta,NaN,16.0,3.0
treatmentb,2.0,11.0,1.0


In [0]:
# Take the row index, and add it as a new column
table1 = table1.reset_index()
table1

,index,treatmenta,treatmentb
0,John Smith,NaN,2
1,Jane Doe,16.0,11
2,Mary Johnson,3.0,1


In [0]:
# What is the unique identifier for each row
# Where is the data at that I want to be in my single "tidy" column
tidy1 = table1.melt(id_vars='index', value_vars=['treatmenta', 'treatmentb'])
tidy1

,index,variable,value
0,John Smith,treatmenta,NaN
1,Jane Doe,treatmenta,16.0
2,Mary Johnson,treatmenta,3.0
3,John Smith,treatmentb,2.0
4,Jane Doe,treatmentb,11.0
5,Mary Johnson,treatmentb,1.0


In [0]:
tidy1 = tidy1.rename(columns={
    'index': 'name', 
    'variable': 'trt', 
    'value': 'result'
})
tidy1

,name,trt,result
0,John Smith,treatmenta,NaN
1,Jane Doe,treatmenta,16.0
2,Mary Johnson,treatmenta,3.0
3,John Smith,treatmentb,2.0
4,Jane Doe,treatmentb,11.0
5,Mary Johnson,treatmentb,1.0


In [0]:
tidy1.trt = tidy1.trt.str.replace('treatment', '')

tidy1

,name,trt,result
0,John Smith,a,NaN
1,Jane Doe,a,16.0
2,Mary Johnson,a,3.0
3,John Smith,b,2.0
4,Jane Doe,b,11.0
5,Mary Johnson,b,1.0


#### Table 2 --> Tidy

In [0]:
table2

,John Smith,Jane Doe,Mary Johnson
treatmenta,NaN,16.0,3.0
treatmentb,2.0,11.0,1.0


In [0]:
##### LEAVE BLANK --an assignment exercise #####

# no transposing allowed in your answer to this question
# don't do this: 
# table2.T.melt(id_vars='index', value_vars=['treatmenta', 'treatmentb'])
tidy2 = table2.pivot_table(columns=['treatmenta', 'treatmentb'])
tidy2 = tidy2.reset_index()
tidy2.columns = ['index', 'a', 'b']
tidy2

,index,a,b
0,Jane Doe,16.0,11.0
1,John Smith,NaN,2.0
2,Mary Johnson,3.0,1.0


In [0]:
tidy2 = tidy2.melt(id_vars='index', value_vars=['a', 'b'])
tidy2

,index,variable,value
0,Jane Doe,a,16.0
1,John Smith,a,NaN
2,Mary Johnson,a,3.0
3,Jane Doe,b,11.0
4,John Smith,b,2.0
5,Mary Johnson,b,1.0


In [0]:
tidy2 = tidy2.rename(columns={
    'index': 'name', 
    'variable': 'trt', 
    'value': 'result'
})
tidy2
# All Tidied Up

,name,trt,result
0,Jane Doe,a,16.0
1,John Smith,a,NaN
2,Mary Johnson,a,3.0
3,Jane Doe,b,11.0
4,John Smith,b,2.0
5,Mary Johnson,b,1.0


### Wide

In [111]:
tidy1

,name,trt,result
0,John Smith,a,NaN
1,Jane Doe,a,16.0
2,Mary Johnson,a,3.0
3,John Smith,b,2.0
4,Jane Doe,b,11.0
5,Mary Johnson,b,1.0


In [113]:
# index: unique identifier
# columns: What do you want to differentiate the columns in wide format
# values: Where are the numbers at - go in the middle of the wide dataframe
wide = tidy1.pivot_table(index='name', columns='trt', values='result')
wide

trt,a,b
name,,
Jane Doe,16.0,11.0
John Smith,NaN,2.0
Mary Johnson,3.0,1.0


#### Tidy -> Table 2

In [115]:
##### LEAVE BLANK --an assignment exercise #####
# back to wide
table2b = tidy2.copy()
table2b = table2b.pivot_table(index='trt', columns='name', values='result')
table2b

name,Jane Doe,John Smith,Mary Johnson
trt,,,
a,16.0,NaN,3.0
b,11.0,2.0,1.0


##### Load seaborn's flights dataset by running the cell below. Then create a pivot table showing the number of passengers by month and year. Use year for the index and month for the columns. You've done it right if you get 112 passengers for January 1949 and 432 passengers for December 1960.

In [0]:
flights = sns.load_dataset('flights')

In [120]:
##### YOUR CODE HERE #####
flyboy = flights.pivot_table(index='year', columns='month')
flyboy

passengers                       ...                                    
month    January February March April  ... September October November December
year                                   ...                                    
1949         112      118   132   129  ...       136     119      104      118
1950         115      126   141   135  ...       158     133      114      140
1951         145      150   178   163  ...       184     162      146      166
1952         171      180   193   181  ...       209     191      172      194
1953         196      196   236   235  ...       237     211      180      201
1954         204      188   235   227  ...       259     229      203      229
1955         242      233   267   269  ...       312     274      237      278
1956         284      277   317   313  ...       355     306      271      306
1957         315      301   356   348  ...       404     347      305      336
1958         340      318   362   348  ...       404     359      310      337
1959         360      342   406   396  ...       463     407      362      405
1960         417      391   419   461  ...       508     461      390      432

[12 rows x 12 columns]

## Join Data Stretch Challenge

The [Instacart blog post](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2) has a visualization of "**Popular products** purchased earliest in the day (green) and latest in the day (red)." 

The post says,

> "We can also see the time of day that users purchase specific products.

> Healthier snacks and staples tend to be purchased earlier in the day, whereas ice cream (especially Half Baked and The Tonight Dough) are far more popular when customers are ordering in the evening.

> **In fact, of the top 25 latest ordered products, the first 24 are ice cream! The last one, of course, is a frozen pizza.**"

Your challenge is to reproduce the list of the top 25 latest ordered popular products.

We'll define "popular products" as products with more than 2,900 orders.



In [0]:
##### YOUR CODE HERE #####

## Reshape Data Stretch Challenge

_Try whatever sounds most interesting to you!_

- Replicate more of Instacart's visualization showing "Hour of Day Ordered" vs "Percent of Orders by Product"
- Replicate parts of the other visualization from [Instacart's blog post](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2), showing "Number of Purchases" vs "Percent Reorder Purchases"
- Get the most recent order for each user in Instacart's dataset. This is a useful baseline when [predicting a user's next order](https://www.kaggle.com/c/instacart-market-basket-analysis)
- Replicate parts of the blog post linked at the top of this notebook: [Modern Pandas, Part 5: Tidy Data](https://tomaugspurger.github.io/modern-5-tidy.html)

In [0]:
##### YOUR CODE HERE #####